# Comprobaciones de versiones instaladas

In [1]:
!python --version

Python 3.8.13


In [2]:
!java -version

openjdk version "1.8.0_362"
OpenJDK Runtime Environment (build 1.8.0_362-8u372-ga~us1-0ubuntu1~22.04-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)


In [3]:
!find / -iname spark*.jar 2>/dev/null|grep sql

/opt/conda/lib/python3.8/site-packages/pyspark/jars/spark-sql_2.12-3.3.1.jar
/usr/local/spark-3.3.1-bin-hadoop3/jars/spark-sql_2.12-3.3.1.jar


In [4]:
import os
os.environ["AWS_REGION"]
#os.environ["AWS_ACCESS_KEY_ID"]

'us-east-1'

In [5]:
!pip install pyspark==3.3.1

# Iniciar conexión spark

In [6]:
import os
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark import SQLContext

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4 pyspark-shell'
dst_lh_url  = os.environ.get('DST_LH_URL', 'spark://spark-master-0.spark-headless.demml.svc.cluster.local:7077')
dst_lh_appn = os.environ.get('DST_LH_APPN', 'conexion_pyspark_iceberg_edu')
import subprocess
my_pod_ip = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip(' \n\t')
my_pod_ip


'172.21.174.71'

In [7]:
spark = SparkSession.builder \
    .master(dst_lh_url) \
    .appName(dst_lh_appn) \
    .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257,software.amazon.awssdk:s3:2.17.257,software.amazon.awssdk:iam:2.17.257,org.apache.hadoop:hadoop-aws:3.2.3') \
    .config('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.sql.catalog.spark_catalog','org.apache.iceberg.spark.SparkSessionCatalog') \
    .config('spark.sql.catalog.spark_catalog.catalog-impl','org.apache.iceberg.jdbc.JdbcCatalog') \
    .config('spark.sql.catalog.spark_catalog.uri','jdbc:postgresql://postgresql:5432/iceberg') \
    .config('spark.driver.host', my_pod_ip) \
    .config('spark.sql.catalog.spark_catalog.jdbc.useSSL','false') \
    .config('spark.sql.catalog.spark_catalog.jdbc.user','iceberg') \
    .config('spark.sql.catalog.spark_catalog.jdbc.password','iceberg') \
    .config('spark.sql.catalog.spark_catalog.warehouse','s3a://warehouse/') \
    .config('spark.sql.catalog.spark_catalog.s3.endpoint','http://minio:9000') \
    .config('spark.sql.catalog.spark_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.sql.defaultCatalog','spark_catalog') \
    .config('spark.hadoop.fs.s3a.endpoint','http://minio:9000') \
    .config('spark.hadoop.fs.s3a.access.key','admin') \
    .config('spark.hadoop.fs.s3a.secret.key','t4bl4red0nd4') \
    .config('spark.hadoop.fs.s3a.path.style.access','true') \
    .config('spark.hadoop.fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()
spark

In [8]:
spark.sql("show current namespace").show() 

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [9]:
spark.sql("show databases").show() 

+---------+
|namespace|
+---------+
|  default|
+---------+



In [31]:
spark.sql("show tables").show() 

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



# Creación de tabla desde spark en minio (Iceberg)

!!! NO borrar siguiente celda comentada ¡¡¡

In [12]:
#spark.sql("CREATE TABLE default.IRIS_EDU_V3 (sepal_length double, sepal_width double, petal_length double, petal_width double, variety string, target int) USING iceberg OPTIONS ('write.object-storage.enabled'=true, 'write.data.path'='s3://warehouse/');").show()

In [32]:
spark.sql("SELECT * FROM spark_catalog.default.IRIS_EDU_V3").show()

+------------+-----------+------------+-----------+-------+------+
|sepal_length|sepal_width|petal_length|petal_width|variety|target|
+------------+-----------+------------+-----------+-------+------+
|         5.1|        3.5|         1.4|        0.2| setosa|     0|
|         4.9|        3.0|         1.4|        0.2| setosa|     0|
|         4.7|        3.2|         1.3|        0.2| setosa|     0|
|         4.6|        3.1|         1.5|        0.2| setosa|     0|
|         5.0|        3.6|         1.4|        0.2| setosa|     0|
|         5.4|        3.9|         1.7|        0.4| setosa|     0|
|         4.6|        3.4|         1.4|        0.3| setosa|     0|
|         5.0|        3.4|         1.5|        0.2| setosa|     0|
|         4.4|        2.9|         1.4|        0.2| setosa|     0|
|         4.9|        3.1|         1.5|        0.1| setosa|     0|
|         5.4|        3.7|         1.5|        0.2| setosa|     0|
|         4.8|        3.4|         1.6|        0.2| setosa|   

# Creo un dataframe de pyspark a partir de un dataframe de pandas y lo escribo en iceberg (se puede ver en minio) 

In [19]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['variety'] = iris.target_names[iris.target]
df['target'] = iris.target

df.columns = ['sepal_length','sepal_width','petal_length','petal_width','variety','target']

In [20]:
dfs = spark.createDataFrame(df)
dfs.show(5)

/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+------------+-----------+------------+-----------+-------+------+
|sepal_length|sepal_width|petal_length|petal_width|variety|target|
+------------+-----------+------------+-----------+-------+------+
|         5.1|        3.5|         1.4|        0.2| setosa|     0|
|         4.9|        3.0|         1.4|        0.2| setosa|     0|
|         4.7|        3.2|         1.3|        0.2| setosa|     0|
|         4.6|        3.1|         1.5|        0.2| setosa|     0|
|         5.0|        3.6|         1.4|        0.2| setosa|     0|
+------------+-----------+------------+-----------+-------+------+
only showing top 5 rows



In [29]:
dfs.write.format("iceberg").mode("append").save("default.IRIS_EDU_V3")

In [30]:
# spark.sql("SELECT * FROM spark_catalog.default.IRIS_EDU_V3").show()
dfs2 = spark.table("spark_catalog.default.IRIS_EDU_V3")
dfs2.show(5)

+------------+-----------+------------+-----------+-------+------+
|sepal_length|sepal_width|petal_length|petal_width|variety|target|
+------------+-----------+------------+-----------+-------+------+
|         5.1|        3.5|         1.4|        0.2| setosa|     0|
|         4.9|        3.0|         1.4|        0.2| setosa|     0|
|         4.7|        3.2|         1.3|        0.2| setosa|     0|
|         4.6|        3.1|         1.5|        0.2| setosa|     0|
|         5.0|        3.6|         1.4|        0.2| setosa|     0|
|         5.4|        3.9|         1.7|        0.4| setosa|     0|
|         4.6|        3.4|         1.4|        0.3| setosa|     0|
|         5.0|        3.4|         1.5|        0.2| setosa|     0|
|         4.4|        2.9|         1.4|        0.2| setosa|     0|
|         4.9|        3.1|         1.5|        0.1| setosa|     0|
|         5.4|        3.7|         1.5|        0.2| setosa|     0|
|         4.8|        3.4|         1.6|        0.2| setosa|   

In [27]:
%load_ext kedro.extras.extensions.ipython
%reload_kedro

catalog.list()
# catalog.load('data_raw_minio')
# catalog.load('data_raw_iceberg')

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython


[09/18/23 08:20:58] INFO     Resolved project path as: /home/coder/eromero/iris-pyspark.            ]8;id=613365;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=966272;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/18/23 08:20:58] WARNING  /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/io ]8;id=73100;file:///home/coder/miniconda3/envs/demml/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=46495;file:///home/coder/miniconda3/envs/demml/lib/python3.8/warnings.py#109\109]8;;\
                             /data_catalog.py:300: FutureWarning: Defining the 'layer' attribute at                
                             the top level is deprecated and will be removed in Kedro 0.19.0.                      
                             Please move 'layer' inside the 'metadata' -> 'kedro-viz' attributes.                  
                             See                                                                                   
                             https://docs.kedro.org/en/latest/visualisation/kedro-viz_visualisation                
                             .html#visualise-layers for more information.                                          
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Kedro project iris-pyspark                                             ]8;id=793589;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=878553;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=556703;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=890302;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=816864;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=806751;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#115\115]8;;\

['data_raw_minio', 'parameters', 'params:example_test_data_ratio', 'params:example_num_trees']

In [28]:
spark

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 spark                                                                                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'spark' is not defined